In [3]:
#| default_exp lhs

%load_ext autoreload
%autoreload 2

In [7]:
# %pip install lark
# %pip install networkx
from lark import Lark

  Using cached lark-1.1.5-py3-none-any.whl (107 kB)
Note: you may need to restart the kernel to use updated packages.
  Using cached networkx-3.0-py3-none-any.whl (2.0 MB)
Note: you may need to restart the kernel to use updated packages.


## LHS parser ##

parsing of the pattern sent as lhs, into a networkX graph representing the template to search.

The module converts the declerative constraints regarding the properties of the nodes and edges in the LHS, to imperative functions that are checked together with the 'condition' parameter

## grammar

In [ ]:
#| hide
#attributes: allow optional \n here, for imperative syntax)
#attribute: #["=" value] 

#    attr_name: /[a-zA-Z0-9]+/ #TODO: lark-imported
#    type:  "int" | "string" | "bool" #TODO: escaped string or word
#    value: /[0-9a-zA-Z]/

#    %import common.WS #CHANGE to allow \n in the imperative option.

In [ ]:
#| export
lhs_parser = Lark(r"""
    %import common.NUMBER -> NATURAL_NUMBER 
    %import common.ESCAPED_STRING
    %import common.WS 
    %ignore WS

    NAMED_VERTEX: /[a-zA-Z0-9]+/ 
    ANONYMUS: "_"
    ATTR_NAME: /[a-zA-Z0-9]+/ 
    TYPE:  "int" | "string" | "bool" 
    VALUE: /[0-9a-zA-Z]/

    attribute: ATTR_NAME [":" TYPE] ["=" VALUE] 
    attributes: "\[" attribute ("," attribute)* "\]"

    multi_connection: "-" NATURAL_NUMBER "+" [attributes] "->" 
                    | "-" NATURAL_NUMBER [attributes] "->" 
    connection: "-" [attributes "-"] ">"
              | multi_connection
    
    index_vertex: NAMED_VERTEX "<" NATURAL_NUMBER ("," NATURAL_NUMBER)* ">"

    vertex: NAMED_VERTEX [attributes]
        | index_vertex [attributes]
        | ANONYMUS [attributes]

    pattern: vertex (connection vertex)*
    patterns: pattern (";" pattern)*
        
    """, parser="lalr", start='patterns' , debug=True)

: 

In [61]:
tree = lhs_parser.parse("aaaaa->b") #.pretty(indent_str = " $ ")
assert(tree != None)

In [62]:
# print(tree)
# assert(tree == 
#     Tree(Token('RULE', 'patterns'),[
#       Tree(Token('RULE', 'pattern'),[
#         Tree(Token('RULE', 'vertex'),[
#           Tree(Token('RULE', 'vertex'),[]),
#           None
#         ])
#       ])
#     ]))

Tree(Token('RULE', 'patterns'), [Tree(Token('RULE', 'pattern'), [Tree(Token('RULE', 'vertex'), [Token('NAMED_VERTEX', 'aaaaa'), None]), Tree(Token('RULE', 'connection'), [None]), Tree(Token('RULE', 'vertex'), [Token('NAMED_VERTEX', 'b'), None])])])


## Transformer
The transformer is designed to return the networkX graph representing the patterns.

For each branch, the appropriate method will be called with the children of the branch as its argument, and its return value will replace the branch in the tree.

In [ ]:
#| export
import itertools
import copy
import networkx as nx

In [ ]:
#| export
cnt:int = 0 # unique id for anonymous vertices
from lark import Tree, Transformer
class lhsTransformer(Transformer):
    def NATURAL_NUMBER(self, number):
        return int(number)
    
    def attribute(self, attr_name, _):
        return (attr_name, "default") # constraints are handled in other transformer.
    
    def attributes(self, *attributes: list):
        # return a packed list of the attribute names.
        attr_dict = {}
        for attribute in attributes:
            attr_dict[attribute[0]] = attribute[1]
        return attr_dict

    def multi_connection(self, number, attributes: dict): # +
        # renewed: return the list of attributes(strings), add a special attribute to denote number of duplications,
        #   and FALSE (indicating that the connection is not deterministic)
        attributes["$dup"] = number
        return (attributes, False)
    
    def multi_connection(self, number, attributes: dict): # no +
        # renewed: return the list of attributes(strings), add a special attribute to denote number of duplications,
        #   and TRUE (indicating that the connection is not deterministic)
        attributes["$dup"] = number
        return (attributes, True)

    def connection(self, multiconnection_params: tuple): #multiconnection
        # return the packed list of attributes received, num_duplications, is_deterministic
        return multiconnection_params

    def connection(self, *attributes): #
        # return the packed list of attributes received, num_duplications = 1, is_deterministic = True
        attributes["$dup"] = 1
        return (attributes, True)

    def ANONYMUS(self): #
        # return a dedicated name for anonymus (string), and an empty list.
        cnt += 1
        return ("$" + str(cnt), [])

    def index_vertex(self, main_name, *numbers):
        # return the main name of the vertex, and a list of the indices specified.
        return (main_name, numbers)
    
    def NAMED_VERTEX(self, name):
        # return the main name of the vertex, and an empty list.
        return (name, [])

    def vertex(self, vertex_tuple: tuple, attributes: dict):
        # return arguments
        name, indices_list = vertex_tuple
        new_name = ",".join(indices_list) # numbers are strings, no convertion needed.
        return (new_name, attributes)

    def pattern(self, vertex, *rest):
        # 1) unpack lists of vertices and connections.
        conn, vertices = rest[::2], rest[1::2]
        vertices.insert(0,vertex)
        # 2) create a networkX graph:
            # if there is a special attribute with TRUE, dumplicate the connection __number__ times.
        G = nx.Graph()

        # simplified vertion - ignore duplications
        G.add_nodes_from(vertices)
        edge_list = []
        for i,edge in enumerate(conn):
            edge_list.append((vertices[i], vertices[i+1], edge[0])) # ignore edge[1] - determinism flag

        # more complex vertion - duplications
        # create a recursive function that adds the vertices and edges, 
        # that calls itself by the number of duplications on each level.

    def patterns(self,):
        g = nx.Graph()
        pass

In [4]:
# import networkx as nx
# H = nx.path_graph(10)
# H.edges


EdgeView([(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9)])

## Type and constant value checking
The transformer is designed to collect the node type and constant node value constraints, such that they are added to the 'condition' parameter to be checked later.

This transformer works on a copy of the tree to keep it intact.

In [ ]:
#| export
class collectTypeConstraints(Transformer):
    def attribute(self, attr_name, type, value):
        # return a mapping from attr_name - > required type and value
        pass

    def attributes(self, *attributes):
        # return a packed list of the attribute mappings.
        pass

    def vertex(self, name, indices_list, attributes_list):
        # same as lhsTransformer
        pass

    def pattern(self, vertex, *connections_to_vertex):
        # return arguments
        pass

    def patterns(self, *patterns):
        # unpack lists of vertices and connections.
        def typeCondition(Match):
            # for every vertex in vertex list:
                # create full_vertex_name by the attached indices list
                # for every attr, type, name required for the vertex:
                    # constructor = getName(type) - get the constructor for the type
                    # 1) check that the required type and value match together.
                    # try:
                    #     instance = constructor(value)
                    # Except:
                        # flag = False: value does not match the type.

                    # 2) check that the value constraint holds
                    # if getattr(instance, __eq__) == None:
                        # flag = False. the type must implement __eq__
                    # if not (instance == match[full_vertex_name][attr])

                    # no need to check the type constraint(?), if the value fits. (python)

            # TODO: perform the same iterations in the connections list.

            #return flag and condition(Match)
            pass

        return typeCondition #sent as a module output and replaces condition.
        pass

Apply the Transformers

In [ ]:
#| export
def parse_lhs(lhs: string):
    tree = lhs_parser.parse(lhs)
    final_g = lhsTransformer().transform(tree) #networkx graph
    return final_g

In [ ]:
required_syntax =  """
a -> b

a -[x:int = ...]-> b

a -> b[x:int = ...]

a -> b -6+[weight:int]-> c -> d[value:int]
d<0> -> e
d<5> -> e

b -+-> d[value:int]
d<0> -7-> e
e<0,5> -> _

b[ \
value: str = \"hello\", \
id: int \
]

b -[
...
]-> c 

"""